# pardigm_control.ipynb
2020/03/24 wi

---
### Step 1. Initialize PCBox

In [ ]:
import serial, time
import Sentech_camera_control as ca

arduino = serial.Serial('COM3', 115200, timeout=.1)

def commu_arduino(commandStr):
    commandStr = commandStr + "\n"
    arduino.write(commandStr.encode()) # Need to encode bytes literal
    time.sleep(0.1)

    msg = arduino.read(arduino.inWaiting()) # read everything in the input buffer
    if msg.decode("utf-8") != "":
        _msg = msg.decode("utf-8")
        print (_msg.replace("\r", ""))
        # print (_msg[:-2]) # Need to decode to string
        # print (_msg.replace("\r", "").encode())
def volume_set(volume):
    loop = int((202 - volume)/2)
    
    commu_arduino("v") # Issue independent command to wait to enter volume adjust mode
    for x in range(loop):
        commu_arduino("-")
    commu_arduino("x")


# Opening welcome
time.sleep(3) #give the connection a second to settle
commu_arduino("")
commu_arduino("set_verbose -off")

In [ ]:
arduino.close()

### Step 2. Set paradigm time table

In [ ]:
# 4 pairing
commu_arduino("set_paradigm -clear")
commu_arduino("set_paradigm -paradigm_name 4_pairing")
commu_arduino("set_paradigm -paradigm_duration 540.0")
commu_arduino("set_paradigm -tone -epoch 0 -start 120.0 -end 150.0")
commu_arduino("set_paradigm -tone -epoch 1 -start 210.0 -end 240.0")
commu_arduino("set_paradigm -tone -epoch 2 -start 360.0 -end 390.0")
commu_arduino("set_paradigm -tone -epoch 3 -start 480.0 -end 510.0")
commu_arduino("set_paradigm -shocker -epoch 0 -start 149.5 -end 150.0")
commu_arduino("set_paradigm -shocker -epoch 1 -start 239.5 -end 240.0")
commu_arduino("set_paradigm -shocker -epoch 2 -start 389.5 -end 390.0")
commu_arduino("set_paradigm -shocker -epoch 3 -start 509.5 -end 510.0")
commu_arduino("set_paradigm -list")

In [ ]:
# Test for Cue
commu_arduino("set_paradigm -clear")
commu_arduino("set_paradigm -paradigm_name Test_for_Cue")
commu_arduino("set_paradigm -paradigm_duration 180.0")
commu_arduino("set_paradigm -tone -epoch 0 -start 60.0 -end 180.0")
commu_arduino("set_paradigm -list")

In [ ]:
# Short paradigm for debug
commu_arduino("set_paradigm -clear")
commu_arduino("set_paradigm -paradigm_name Short_paradigm_for_debug")
commu_arduino("set_paradigm -paradigm_duration 15.0")
commu_arduino("set_paradigm -tone -epoch 0 -start 5.0 -end 8.0")
commu_arduino("set_paradigm -shocker -epoch 0 -start 7.5 -end 10.0")
commu_arduino("set_paradigm -list")

In [ ]:
# reset paradigm table
commu_arduino("set_paradigm -clear")
commu_arduino("set_paradigm -list")

### Step 3. Adjust tone volume
    set_tone -tone_volume 30 [range:0-50]
    set_tone -UART

In [ ]:
tone_volume = 50

# Do not modify the followings
commu_arduino("set_tone -tone_volume " + str(tone_volume))

### Step 4. Adjust camera frame rate (fps), exposure time (ms) and focus.
    Defaults: 33 fps (frame interval: 30 ms), exposeure time: 20 ms

#### Start camera
    Set cameraNum

In [ ]:
cameraNum = 3

# Do not modify the following
savePath = r''  # '' means not to save movie
fps = 5         # fps = 5 is dummy
commu_arduino("set_camera -cameras_number " + str(cameraNum))
commu_arduino("set_camera -start")
ca.live_movie(cameraNum, fps, savePath)

#### Adjust fps and exposure time (ms)
    Set fps and exposure

In [ ]:
fps = 5
exposure = 10

# Do not modify the followings
commu_arduino("set_camera -fps " + str(fps) + " -exposure " + str(exposure))
commu_arduino("set_camera -status")

#### Stop cameras
    Preview windows will automatically close after 5 timeout.

In [ ]:
commu_arduino("set_camera -stop")

### Step 5. Start paradigm

#### Output file which document the paradigm condition
    Set output directory (savePath)
    You need to create the output directory in advance.

In [ ]:
savePath = r'C:\Users\User\Desktop\project'

# Do not modify the followings
import datetime
import os

commu_arduino("")
commu_arduino("")

x = datetime.datetime.now()
savePath += x.strftime("\\%Y%m%d-%H%M%S")

if not os.path.exists(savePath):
    os.mkdir(savePath)

prefixFileName = os.path.split(savePath)[1]
outputFileName = os.path.join(savePath, prefixFileName + "_" + "output.txt")

import sys

orig_stdout = sys.stdout
f = open(outputFileName, 'w')
sys.stdout = f

print("1. savePath:          " + savePath)
print("2. date:              " + str(datetime.datetime.now()))
print("3. tone volume:       " + str(tone_volume))
print("4. camera settings:   ")
commu_arduino("set_camera -status")
print("5. paradigm table:    ")
commu_arduino("set_paradigm -list")

sys.stdout = orig_stdout
f.close()

ca.live_movie(cameraNum, fps, savePath)
commu_arduino("start paradigm")

while 1:
    time.sleep(1) #give the connection a second to settle
    commu_arduino("")

### Step 6. Disconnect PCBox

In [ ]:
arduino.close()